## Preskúmajte K-Means zhlukovanie pomocou R a princípov Tidy dát.

### [**Kvíz pred prednáškou**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/29/)

V tejto lekcii sa naučíte, ako vytvárať zhluky pomocou balíka Tidymodels a ďalších balíkov v ekosystéme R (nazveme ich priatelia 🧑‍🤝‍🧑) a datasetu nigérijskej hudby, ktorý ste importovali skôr. Pokryjeme základy K-Means pre zhlukovanie. Majte na pamäti, že ako ste sa naučili v predchádzajúcej lekcii, existuje mnoho spôsobov, ako pracovať so zhlukmi, a metóda, ktorú použijete, závisí od vašich dát. Skúsime K-Means, pretože je to najbežnejšia technika zhlukovania. Poďme na to!

Pojmy, o ktorých sa dozviete:

-   Silhouette skórovanie

-   Metóda lakťa

-   Inercia

-   Variancia

### **Úvod**

[K-Means zhlukovanie](https://wikipedia.org/wiki/K-means_clustering) je metóda odvodená z oblasti spracovania signálov. Používa sa na rozdelenie a rozčlenenie skupín dát do `k zhlukov` na základe podobností ich vlastností.

Zhluky môžu byť vizualizované ako [Voronoi diagramy](https://wikipedia.org/wiki/Voronoi_diagram), ktoré zahŕňajú bod (alebo 'semienko') a jeho zodpovedajúcu oblasť.

<p >
   <img src="../../images/voronoi.png"
   width="500"/>
   <figcaption>Infografika od Jen Looper</figcaption>


K-Means zhlukovanie má nasledujúce kroky:

1.  Data scientist začne tým, že špecifikuje požadovaný počet zhlukov, ktoré sa majú vytvoriť.

2.  Následne algoritmus náhodne vyberie K pozorovaní z datasetu, ktoré budú slúžiť ako počiatočné centrá zhlukov (t.j. centroidy).

3.  Potom sa každé zvyšné pozorovanie priradí k najbližšiemu centroidu.

4.  Následne sa vypočíta nový priemer každého zhluku a centroid sa presunie na tento priemer.

5.  Teraz, keď boli centrá prepočítané, každé pozorovanie sa opäť skontroluje, či by nemohlo byť bližšie k inému zhluku. Všetky objekty sa znovu priradia pomocou aktualizovaných priemerov zhlukov. Kroky priraďovania zhlukov a aktualizácie centroidov sa iteratívne opakujú, kým sa priradenia zhlukov prestanú meniť (t.j. keď sa dosiahne konvergencia). Typicky algoritmus končí, keď každá nová iterácia vedie k zanedbateľnému pohybu centroidov a zhluky sa stanú statickými.

<div>

> Upozorňujeme, že kvôli náhodnosti počiatočných k pozorovaní použitých ako východiskové centroidy môžeme získať mierne odlišné výsledky pri každom použití postupu. Z tohto dôvodu väčšina algoritmov používa niekoľko *náhodných začiatkov* a vyberie iteráciu s najnižším WCSS. Preto sa dôrazne odporúča vždy spustiť K-Means s viacerými hodnotami *nstart*, aby sa predišlo *nežiaducemu lokálnemu optimu.*

</div>

Táto krátka animácia využívajúca [ilustrácie](https://github.com/allisonhorst/stats-illustrations) od Allison Horst vysvetľuje proces zhlukovania:

<p >
   <img src="../../images/kmeans.gif"
   width="550"/>
   <figcaption>Ilustrácia od @allison_horst</figcaption>



Základná otázka, ktorá pri zhlukovaní vyvstáva, je táto: ako viete, na koľko zhlukov rozdeliť vaše dáta? Jednou z nevýhod použitia K-Means je skutočnosť, že budete musieť určiť `k`, teda počet `centroidov`. Našťastie `metóda lakťa` pomáha odhadnúť dobrú východiskovú hodnotu pre `k`. Vyskúšate si to o chvíľu.

### 

**Predpoklady**

Nadviažeme presne tam, kde sme skončili v [predchádzajúcej lekcii](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb), kde sme analyzovali dataset, vytvorili množstvo vizualizácií a filtrovali dataset na pozorovania, ktoré nás zaujímajú. Určite si ju pozrite!

Budeme potrebovať niekoľko balíkov na dokončenie tohto modulu. Môžete si ich nainštalovať pomocou: `install.packages(c('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork'))`

Alternatívne, nasledujúci skript skontroluje, či máte balíky potrebné na dokončenie tohto modulu, a nainštaluje ich za vás, ak niektoré chýbajú.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))

pacman::p_load('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork')


Poďme na to!

## 1. Tanec s dátami: Zúžme to na 3 najpopulárnejšie hudobné žánre

Toto je zhrnutie toho, čo sme robili v predchádzajúcej lekcii. Poďme si trochu pohrať s dátami!


In [ ]:
# Load the core tidyverse and make it available in your current R session
library(tidyverse)

# Import the data into a tibble
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/5-Clustering/data/nigerian-songs.csv", show_col_types = FALSE)

# Narrow down to top 3 popular genres
nigerian_songs <- df %>% 
  # Concentrate on top 3 genres
  filter(artist_top_genre %in% c("afro dancehall", "afropop","nigerian pop")) %>% 
  # Remove unclassified observations
  filter(popularity != 0)



# Visualize popular genres using bar plots
theme_set(theme_light())
nigerian_songs %>%
  count(artist_top_genre) %>%
  ggplot(mapping = aes(x = artist_top_genre, y = n,
                       fill = artist_top_genre)) +
  geom_col(alpha = 0.8) +
  paletteer::scale_fill_paletteer_d("ggsci::category10_d3") +
  ggtitle("Top genres") +
  theme(plot.title = element_text(hjust = 0.5))


🤩 To išlo skvele!

## 2. Viac prieskumu údajov.

Aké čisté sú tieto údaje? Poďme skontrolovať odľahlé hodnoty pomocou boxplotov. Zameriame sa na číselné stĺpce s menším počtom odľahlých hodnôt (aj keď by ste mohli odľahlé hodnoty vyčistiť). Boxploty môžu ukázať rozsah údajov a pomôžu pri výbere stĺpcov na použitie. Upozorňujeme, že boxploty neukazujú rozptyl, čo je dôležitý prvok pre dobré zoskupiteľné údaje. Pre ďalšie čítanie si pozrite [túto diskusiu](https://stats.stackexchange.com/questions/91536/deduce-variance-from-boxplot).

[Boxploty](https://en.wikipedia.org/wiki/Box_plot) sa používajú na grafické znázornenie distribúcie `číselných` údajov, takže začnime *výberom* všetkých číselných stĺpcov spolu s populárnymi hudobnými žánrami.


In [ ]:
# Select top genre column and all other numeric columns
df_numeric <- nigerian_songs %>% 
  select(artist_top_genre, where(is.numeric)) 

# Display the data
df_numeric %>% 
  slice_head(n = 5)


Pozrite sa, ako výberový pomocník `where` uľahčuje tento proces 💁. Preskúmajte ďalšie podobné funkcie [tu](https://tidyselect.r-lib.org/).

Keďže budeme vytvárať boxplot pre každú číselnú vlastnosť a chceme sa vyhnúť používaniu slučiek, preformátujme naše údaje do *dlhšieho* formátu, ktorý nám umožní využiť `facets` - podgrafy, z ktorých každý zobrazuje jednu podmnožinu údajov.


In [ ]:
# Pivot data from wide to long
df_numeric_long <- df_numeric %>% 
  pivot_longer(!artist_top_genre, names_to = "feature_names", values_to = "values") 

# Print out data
df_numeric_long %>% 
  slice_head(n = 15)


Oveľa dlhšie! Teraz je čas na nejaké `ggplots`! Aký `geom` použijeme?


In [ ]:
# Make a box plot
df_numeric_long %>% 
  ggplot(mapping = aes(x = feature_names, y = values, fill = feature_names)) +
  geom_boxplot() +
  facet_wrap(~ feature_names, ncol = 4, scales = "free") +
  theme(legend.position = "none")


Jednoduché-gg!

Teraz môžeme vidieť, že tieto údaje sú trochu šumivé: pri pozorovaní jednotlivých stĺpcov ako boxplotov môžete vidieť odľahlé hodnoty. Mohli by ste prejsť dataset a odstrániť tieto odľahlé hodnoty, ale to by údaje dosť zredukovalo.

Zatiaľ si vyberme, ktoré stĺpce použijeme na náš klastrovací cvičenie. Vyberme si číselné stĺpce s podobnými rozsahmi. Mohli by sme zakódovať `artist_top_genre` ako číselnú hodnotu, ale zatiaľ ho vynecháme.


In [ ]:
# Select variables with similar ranges
df_numeric_select <- df_numeric %>% 
  select(popularity, danceability, acousticness, loudness, energy) 

# Normalize data
# df_numeric_select <- scale(df_numeric_select)


## 3. Výpočet k-means zoskupovania v R

K-means môžeme vypočítať v R pomocou zabudovanej funkcie `kmeans`, pozrite si `help("kmeans()")`. Funkcia `kmeans()` prijíma dátový rámec so všetkými číselnými stĺpcami ako svoj hlavný argument.

Prvým krokom pri používaní k-means zoskupovania je určiť počet klastrov (k), ktoré budú vytvorené vo finálnom riešení. Vieme, že v dátovom súbore sme identifikovali 3 hudobné žánre, takže skúsme 3:


In [ ]:
set.seed(2056)
# Kmeans clustering for 3 clusters
kclust <- kmeans(
  df_numeric_select,
  # Specify the number of clusters
  centers = 3,
  # How many random initial configurations
  nstart = 25
)

# Display clustering object
kclust


Objekt kmeans obsahuje niekoľko informácií, ktoré sú dobre vysvetlené v `help("kmeans()")`. Momentálne sa zamerajme na niekoľko z nich. Vidíme, že údaje boli rozdelené do 3 klastrov s veľkosťami 65, 110, 111. Výstup tiež obsahuje centrá klastrov (priemery) pre 3 skupiny naprieč 5 premennými.

Vektor klastrovania je priradenie klastrov pre každé pozorovanie. Použime funkciu `augment`, aby sme pridali priradenie klastrov do pôvodného súboru údajov.


In [ ]:
# Add predicted cluster assignment to data set
augment(kclust, df_numeric_select) %>% 
  relocate(.cluster) %>% 
  slice_head(n = 10)


Perfektné, práve sme rozdelili náš dátový súbor do 3 skupín. Takže, aké dobré je naše zhlukovanie 🤷? Pozrime sa na `Silhouette skóre`.

### **Silhouette skóre**

[Silhouette analýza](https://en.wikipedia.org/wiki/Silhouette_(clustering)) môže byť použitá na štúdium vzdialenosti medzi výslednými zhlukmi. Toto skóre sa pohybuje od -1 do 1, pričom skóre blízke 1 znamená, že zhluk je hustý a dobre oddelený od ostatných zhlukov. Hodnota blízka 0 predstavuje prekrývajúce sa zhluky so vzorkami veľmi blízko rozhodovacej hranice susedných zhlukov. [zdroj](https://dzone.com/articles/kmeans-silhouette-score-explained-with-python-exam).

Metóda priemerného silhouette skóre vypočíta priemerné silhouette skóre pozorovaní pre rôzne hodnoty *k*. Vysoké priemerné silhouette skóre naznačuje dobré zhlukovanie.

Funkcia `silhouette` v balíku cluster sa používa na výpočet priemernej šírky silhouette.

> Silhouette môže byť vypočítané s akoukoľvek [vzdialenosťou](https://en.wikipedia.org/wiki/Distance "Distance"), ako napríklad [Euklidovská vzdialenosť](https://en.wikipedia.org/wiki/Euclidean_distance "Euclidean distance") alebo [Manhattanská vzdialenosť](https://en.wikipedia.org/wiki/Manhattan_distance "Manhattan distance"), o ktorých sme hovorili v [predchádzajúcej lekcii](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb).


In [ ]:
# Load cluster package
library(cluster)

# Compute average silhouette score
ss <- silhouette(kclust$cluster,
                 # Compute euclidean distance
                 dist = dist(df_numeric_select))
mean(ss[, 3])


Náš skóre je **0,549**, teda presne v strede. To naznačuje, že naše údaje nie sú obzvlášť vhodné pre tento typ zoskupovania. Pozrime sa, či môžeme tento predpoklad vizuálne potvrdiť. Balík [factoextra](https://rpkgs.datanovia.com/factoextra/index.html) poskytuje funkcie (`fviz_cluster()`) na vizualizáciu zoskupovania.


In [ ]:
library(factoextra)

# Visualize clustering results
fviz_cluster(kclust, df_numeric_select)


Prekrývanie klastrov naznačuje, že naše údaje nie sú obzvlášť vhodné pre tento typ klastrovania, ale poďme pokračovať.

## 4. Určenie optimálneho počtu klastrov

Základná otázka, ktorá často vyvstáva pri K-Means klastrovaní, je táto - bez známych triednych označení, ako zistíte, na koľko klastrov rozdeliť vaše údaje?

Jedným zo spôsobov, ako to môžeme zistiť, je použiť vzorku údajov na `vytvorenie série modelov klastrovania` s narastajúcim počtom klastrov (napr. od 1 do 10) a vyhodnotiť metriky klastrovania, ako je **Silhouette skóre.**

Určme optimálny počet klastrov výpočtom algoritmu klastrovania pre rôzne hodnoty *k* a vyhodnotením **Súčtu štvorcov v rámci klastrov** (WCSS). Celkový súčet štvorcov v rámci klastrov (WCSS) meria kompaktnosť klastrovania a chceme, aby bol čo najmenší, pričom nižšie hodnoty znamenajú, že dátové body sú bližšie.

Preskúmajme vplyv rôznych volieb `k`, od 1 do 10, na toto klastrovanie.


In [ ]:
# Create a series of clustering models
kclusts <- tibble(k = 1:10) %>% 
  # Perform kmeans clustering for 1,2,3 ... ,10 clusters
  mutate(model = map(k, ~ kmeans(df_numeric_select, centers = .x, nstart = 25)),
  # Farm out clustering metrics eg WCSS
         glanced = map(model, ~ glance(.x))) %>% 
  unnest(cols = glanced)
  

# View clustering rsulsts
kclusts


Teraz, keď máme celkový súčet štvorcov v rámci klastrov (tot.withinss) pre každý algoritmus zhlukovania s centrom *k*, použijeme [metódu lakťa](https://en.wikipedia.org/wiki/Elbow_method_(clustering)) na nájdenie optimálneho počtu klastrov. Táto metóda spočíva v zobrazení WCSS ako funkcie počtu klastrov a výbere [lakťa krivky](https://en.wikipedia.org/wiki/Elbow_of_the_curve "Elbow of the curve") ako počtu klastrov, ktoré sa majú použiť.


In [ ]:
set.seed(2056)
# Use elbow method to determine optimum number of clusters
kclusts %>% 
  ggplot(mapping = aes(x = k, y = tot.withinss)) +
  geom_line(size = 1.2, alpha = 0.8, color = "#FF7F0EFF") +
  geom_point(size = 2, color = "#FF7F0EFF")


Graf ukazuje výrazné zníženie WCSS (teda väčšiu *súdržnosť*) pri zvýšení počtu klastrov z jedného na dva, a ďalšie výrazné zníženie z dvoch na tri klastre. Potom je zníženie menej výrazné, čo vedie k vytvoreniu `laktu` 💪 na grafe približne pri troch klastroch. Toto je dobrý indikátor, že existujú dva až tri rozumne dobre oddelené klastre dátových bodov.

Teraz môžeme pokračovať a extrahovať model klastrovania, kde `k = 3`:

> `pull()`: používa sa na extrakciu jedného stĺpca
>
> `pluck()`: používa sa na indexovanie dátových štruktúr, ako sú zoznamy


In [ ]:
# Extract k = 3 clustering
final_kmeans <- kclusts %>% 
  filter(k == 3) %>% 
  pull(model) %>% 
  pluck(1)


final_kmeans


Skvelé! Poďme si vizualizovať získané klastre. Máte záujem o interaktivitu pomocou `plotly`?


In [ ]:
# Add predicted cluster assignment to data set
results <-  augment(final_kmeans, df_numeric_select) %>% 
  bind_cols(df_numeric %>% select(artist_top_genre)) 

# Plot cluster assignments
clust_plt <- results %>% 
  ggplot(mapping = aes(x = popularity, y = danceability, color = .cluster, shape = artist_top_genre)) +
  geom_point(size = 2, alpha = 0.8) +
  paletteer::scale_color_paletteer_d("ggthemes::Tableau_10")

ggplotly(clust_plt)


Možno by sme očakávali, že každý klaster (reprezentovaný rôznymi farbami) bude mať odlišné žánre (reprezentované rôznymi tvarmi).

Pozrime sa na presnosť modelu.


In [ ]:
# Assign genres to predefined integers
label_count <- results %>% 
  group_by(artist_top_genre) %>% 
  mutate(id = cur_group_id()) %>% 
  ungroup() %>% 
  summarise(correct_labels = sum(.cluster == id))


# Print results  
cat("Result:", label_count$correct_labels, "out of", nrow(results), "samples were correctly labeled.")

cat("\nAccuracy score:", label_count$correct_labels/nrow(results))


Presnosť tohto modelu nie je zlá, ale ani výborná. Môže to byť spôsobené tým, že dáta nie sú vhodné pre K-Means Clustering. Tieto dáta sú príliš nevyvážené, málo korelované a medzi hodnotami stĺpcov je príliš veľká variabilita na to, aby sa dali dobre zoskupiť. V skutočnosti sú vytvorené klastre pravdepodobne silne ovplyvnené alebo skreslené tromi kategóriami žánrov, ktoré sme definovali vyššie.

Napriek tomu to bol celkom poučný proces!

V dokumentácii Scikit-learn môžete vidieť, že model ako tento, s klastrami, ktoré nie sú veľmi dobre ohraničené, má problém s „variabilitou“:

<p >
   <img src="../../images/problems.png"
   width="500"/>
   <figcaption>Infografika zo Scikit-learn</figcaption>



## **Variabilita**

Variabilita je definovaná ako „priemer štvorcových rozdielov od priemeru“ [zdroj](https://www.mathsisfun.com/data/standard-deviation.html). V kontexte tohto problému zoskupovania sa vzťahuje na dáta, pri ktorých hodnoty v našej dátovej sade majú tendenciu odchýliť sa príliš od priemeru.

✅ Toto je skvelý moment na zamyslenie sa nad všetkými spôsobmi, ako by ste mohli tento problém napraviť. Upraviť dáta trochu viac? Použiť iné stĺpce? Použiť iný algoritmus? Tip: Skúste [škálovať svoje dáta](https://www.mygreatlearning.com/blog/learning-data-science-with-k-means-clustering/) na ich normalizáciu a otestujte iné stĺpce.

> Skúste tento '[kalkulátor variability](https://www.calculatorsoup.com/calculators/statistics/variance-calculator.php)', aby ste lepšie pochopili tento koncept.

------------------------------------------------------------------------

## **🚀Výzva**

Strávte nejaký čas s týmto notebookom a upravujte parametre. Dokážete zlepšiť presnosť modelu tým, že dáta viac vyčistíte (napríklad odstránením odľahlých hodnôt)? Môžete použiť váhy na pridanie väčšej váhy určitým vzorkám dát. Čo ešte môžete urobiť, aby ste vytvorili lepšie klastre?

Tip: Skúste škálovať svoje dáta. V notebooku je komentovaný kód, ktorý pridáva štandardné škálovanie, aby sa stĺpce dát viac podobali z hľadiska rozsahu. Zistíte, že zatiaľ čo skóre siluety klesá, „zlom“ v grafe lakťa sa vyhladzuje. Je to preto, že ponechanie dát neškálovaných umožňuje dátam s menšou variabilitou niesť väčšiu váhu. Prečítajte si o tomto probléme viac [tu](https://stats.stackexchange.com/questions/21222/are-mean-normalization-and-feature-scaling-needed-for-k-means-clustering/21226#21226).

## [**Kvíz po prednáške**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/30/)

## **Recenzia a samostatné štúdium**

-   Pozrite si simulátor K-Means [ako tento](https://user.ceng.metu.edu.tr/~akifakkus/courses/ceng574/k-means/). Tento nástroj môžete použiť na vizualizáciu vzorových dátových bodov a určenie ich centroidov. Môžete upraviť náhodnosť dát, počet klastrov a počet centroidov. Pomáha vám to získať predstavu o tom, ako sa dáta môžu zoskupovať?

-   Pozrite si tiež [tento materiál o K-Means](https://stanford.edu/~cpiech/cs221/handouts/kmeans.html) zo Stanfordu.

Chcete si vyskúšať svoje novo nadobudnuté zručnosti zoskupovania na dátových sadách, ktoré sa dobre hodia pre K-Means clustering? Pozrite si:

-   [Trénovanie a hodnotenie modelov zoskupovania](https://rpubs.com/eR_ic/clustering) pomocou Tidymodels a priateľov

-   [Analýza klastrov K-Means](https://uc-r.github.io/kmeans_clustering), UC Business Analytics R Programming Guide

- [K-Means clustering s princípmi upravených dát](https://www.tidymodels.org/learn/statistics/k-means/)

## **Úloha**

[Skúste rôzne metódy zoskupovania](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/2-K-Means/assignment.md)

## ĎAKUJEME:

[Jen Looper](https://www.twitter.com/jenlooper) za vytvorenie pôvodnej verzie tohto modulu v Pythone ♥️

[`Allison Horst`](https://twitter.com/allison_horst/) za vytvorenie úžasných ilustrácií, ktoré robia R prístupnejším a zábavnejším. Viac ilustrácií nájdete v jej [galérii](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

Šťastné učenie,

[Eric](https://twitter.com/ericntay), Gold Microsoft Learn Student Ambassador.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="500"/>
   <figcaption>Ilustrácia od @allison_horst</figcaption>



---

**Upozornenie**:  
Tento dokument bol preložený pomocou služby AI prekladu [Co-op Translator](https://github.com/Azure/co-op-translator). Hoci sa snažíme o presnosť, prosím, berte na vedomie, že automatizované preklady môžu obsahovať chyby alebo nepresnosti. Pôvodný dokument v jeho rodnom jazyku by mal byť považovaný za autoritatívny zdroj. Pre kritické informácie sa odporúča profesionálny ľudský preklad. Nie sme zodpovední za akékoľvek nedorozumenia alebo nesprávne interpretácie vyplývajúce z použitia tohto prekladu.
